In [ ]:
import mmf_setup;mmf_setup.nbinit(hgroot=False)
from mmfutils.contexts import NoInterrupt
try: from importlib import reload
except ImportError: pass
%pylab inline

# Fulde-Ferrel (FF) States

In [ ]:
import numpy as np
import tf_completion as tf

np.random.seed(1)
m, hbar, kF = 1 + np.random.random(3)
eF = (hbar*kF)**2/2/m
nF = kF**3/3/np.pi**2
mu = 0.59060550703283853378393810185221521748413488992993*eF
delta = 0.68640205206984016444108204356564421137062514068346*eF
args0 = dict(mu_a=mu, mu_b=mu, delta=delta, m_a=m, m_b=m, 
             hbar=hbar, d=3, T=0.0)

ss = np.linspace(-2, 2, 50)
res = []
for s in ss:
    dmu = s*delta
    mu_a, mu_b = mu + dmu, mu - dmu
    args = dict(args0, mu_a=mu_a, mu_b=mu_b)
    n_p = tf.integrate(tf.n_p_integrand, **args)
    n_m = tf.integrate(tf.n_m_integrand, **args)
    res.append((n_p.n, n_m.n))

res = np.asarray(res)
plt.plot(ss, res);plt.xlabel('dmu/delta');plt.ylabel('n')

In [ ]:
from scipy.optimize import brentq
def get_delta(dmu=0, q=0, debug=False, **kw):
    def f(delta):        
        args = dict(args0)
        args['mu_a'], args['mu_b'] = mu + dmu, mu - dmu
        args.update(kw)        
        args['delta'] = delta
        C = tf.integrate(tf.C_integrand, k_c=10.0, **args)
        return C.n
    if debug:
        return f
    return brentq(f, 0.0, 1.0)

ss = np.linspace(0.3, 1.1, 20)
res = []
for s in ss:
    f = get_delta(dmu=s*delta, debug=True)
    res.append(f(0.001))
plt.plot(ss, res);plt.xlabel('dmu/delta');plt.ylabel('C(0)')

In [ ]:
from scipy.optimize import brentq
def get_delta(dmu=0, q=0, debug=False, **kw):
    def f(delta):        
        args = dict(args0, m=args0['m_a'])
        [args.pop(_k, None) for _k in ['m_a', 'm_b']]
        args['mu_a'], args['mu_b'] = mu + dmu, mu - dmu
        args.update(kw)        
        args['delta'] = delta
        C = tf.compute_C(**args)
        return C.n
    if debug:
        return f
    return brentq(f, 0.0, 1.0)

ss = np.linspace(0.3, 1.1, 20)
res = []
for s in ss:
    f = get_delta(dmu=s*delta, debug=True)
    res.append(f(0.001))
plt.plot(ss, res);plt.xlabel('dmu/delta');plt.ylabel('C(0)')

In [ ]:
ss = np.linspace(-4, 4, 100)
res = []
for s in ss:
    args = dict(args0, m=m, delta=s*delta)
    del args['m_a'], args['m_b']
    
    C = tf.compute_C(**args)
    res.append(C.n)

res = np.asarray(res)
plt.plot(ss*delta, res);plt.xlabel('delta');plt.ylabel('C')